In [58]:
import tensorflow as tf
print("Usandor Tensorflow version " + tf.__version__)


if tf.test.gpu_device_name():
  print('Usando GPU: {}'.format(tf.test.gpu_device_name()))
else:
  print("Usando CPU.")

Usandor Tensorflow version 2.13.1
Usando GPU: /device:GPU:0


2023-11-20 10:16:33.547023: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-20 10:16:33.547158: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-20 10:16:33.547212: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [59]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [60]:
NUMBER_OF_LAYERS = [1,2]
NUMBER_OF_NEURONS = [[25,25,25,25],[50,50,50,50],[100,100,100,100],[200,200,200,200], [1000,1000,1000,1000]]
NUMBER_OF_STEPS = [1,2,3,4,5,6,7,8,9,10]

In [61]:
from numpy import array, hstack
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import pandas as pd
from empresa4.datasets import get_dataset, nombres_datasets
from keras.callbacks import EarlyStopping
from datetime import datetime
from empresa4.core import calculate_error_2

In [62]:
# Function to split a single time series into overlapping sequences
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix - 1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [63]:
# Read the data
df = get_dataset("02_productos_todos_anti_leak")
df

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados
0,201701,20001,479.0,937.72717,934.77222,HC,ROPA LAVADO,3000.0,0
1,201701,20002,391.0,555.18654,550.15707,HC,ROPA LAVADO,3000.0,0
2,201701,20003,438.0,1067.81543,1063.45835,FOODS,ADEREZOS,475.0,0
3,201701,20004,339.0,569.37394,555.91614,FOODS,ADEREZOS,240.0,0
4,201701,20005,249.0,494.60084,494.27011,FOODS,ADEREZOS,120.0,0
...,...,...,...,...,...,...,...,...,...
33691,201902,21235,0.0,0.00000,0.00000,PC,PIEL1,200.0,0
33692,201902,21236,0.0,0.00000,0.00000,PC,PIEL1,400.0,0
33693,201902,21115,0.0,0.00000,0.00000,PC,DEOS,89.0,0
33694,201902,20734,0.0,0.00000,0.00000,PC,CABELLO,400.0,0


In [64]:
cluster_df = pd.read_csv("product_ids_clusters.csv")
cluster_df

,product_id,201701_norm,201702_norm,201703_norm,201704_norm,201705_norm,201706_norm,201707_norm,201708_norm,201709_norm,...,201806_norm,201807_norm,201808_norm,201809_norm,201810_norm,201811_norm,201812_norm,201901_norm,201902_norm,cluster
0,20001,0.091342,0.347663,0.567846,0.466153,0.654484,0.662267,0.449035,0.552176,0.573766,...,0.501371,0.640632,0.784656,0.626804,1.000000,0.789908,0.647724,0.555827,0.548559,0
1,20002,0.035110,0.286313,0.472443,0.295634,0.477368,0.547960,0.478475,0.350738,0.602969,...,0.585130,0.553193,0.657610,0.540080,0.780210,1.000000,0.571334,0.716985,0.590329,0
2,20003,0.375239,0.383889,0.468175,0.268330,0.316705,0.379824,0.400778,0.441167,0.748375,...,0.337259,0.334958,0.488019,0.465740,0.670507,0.616162,0.392968,0.492512,0.387094,0
3,20004,0.164645,0.400644,0.386223,0.403683,0.428377,0.465553,0.449286,0.822024,0.993244,...,0.353047,0.515169,0.738344,0.748157,0.638388,0.632612,0.461655,0.403118,0.348204,0
4,20005,0.179476,0.441811,0.451804,0.530902,0.413083,0.423504,0.501455,0.855840,1.000000,...,0.438761,0.402467,0.701053,0.610395,0.716162,0.375956,0.298511,0.291258,0.328378,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,21295,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3
1292,21296,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
1293,21297,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3
1294,21298,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1


In [65]:
# add column cluster from cluster_df to df
df = df.merge(cluster_df[["product_id", "cluster"]], on='product_id', how='left')
df["cluster"].isna().sum()

0

In [66]:
df[df["product_id"] == 20001]

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados,cluster
0,201701,20001,479.0,937.72717,934.77222,HC,ROPA LAVADO,3000.0,0,0
785,201702,20001,432.0,833.72187,798.01620,HC,ROPA LAVADO,3000.0,0,0
1566,201703,20001,509.0,1330.74697,1303.35771,HC,ROPA LAVADO,3000.0,0,0
2352,201704,20001,279.0,1132.94430,1069.96130,HC,ROPA LAVADO,3000.0,0,0
3136,201705,20001,701.0,1550.68936,1502.20132,HC,ROPA LAVADO,3000.0,0,0
3942,201706,20001,570.0,1575.82891,1520.06539,HC,ROPA LAVADO,3000.0,0,0
4765,201707,20001,381.0,1086.47101,1030.67391,HC,ROPA LAVADO,3000.0,0,0
5591,201708,20001,643.0,1289.66869,1267.39462,HC,ROPA LAVADO,3000.0,0,0
6438,201709,20001,381.0,1356.96103,1316.94604,HC,ROPA LAVADO,3000.0,0,0
7267,201710,20001,273.0,1441.60247,1439.75563,HC,ROPA LAVADO,3000.0,0,0


In [67]:
equiv = {
    201701: 201703,
    201702: 201704,
    201703: 201705,
    201704: 201706,
    201705: 201707,
    201706: 201708,
    201707: 201709,
    201708: 201710,
    201709: 201711,
    201710: 201712,
    201711: 201801,
    201712: 201802,
    201801: 201803,
    201802: 201804,
    201803: 201805,
    201804: 201806,
    201805: 201807,
    201806: 201808,
    201807: 201809,
    201808: 201810,
    201809: 201811,
    201810: 201812,
    201811: 201901,
    201812: 201902,
    201901: 201903,
    201902: 201904,
    201903: 201905,
    201904: 201906,
    201905: 201907,
    201906: 201908,
    201907: 201909,
    201908: 201910,
    201909: 201911,
    201910: 201912,
    201911: 202001,
    201912: 202002,
    202001: 202003,
    202002: 202004,
}
target_df = get_dataset("02_productos_todos")


# Filter data up to 201902
def lag_target_class(row):
    # from the column "periodo" and "product_id" of this row, get the equivalen periodo in equiv and get the tn column from "target" df for this product_id and the equiv periodo
    product_id = row["product_id"]
    periodo = row["periodo"]
    periodo_equiv = equiv.get(periodo)
    if periodo_equiv is None:
        return None
    value = target_df[(target_df["product_id"] == product_id) & (target_df["periodo"] == periodo_equiv)]["tn"].values[0]
    return value

In [68]:
df["target"] = df.apply(lag_target_class, axis=1)
df

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados,cluster,target
0,201701,20001,479.0,937.72717,934.77222,HC,ROPA LAVADO,3000.0,0,0,1303.35771
1,201701,20002,391.0,555.18654,550.15707,HC,ROPA LAVADO,3000.0,0,0,834.73521
2,201701,20003,438.0,1067.81543,1063.45835,FOODS,ADEREZOS,475.0,0,0,917.16548
3,201701,20004,339.0,569.37394,555.91614,FOODS,ADEREZOS,240.0,0,0,489.91328
4,201701,20005,249.0,494.60084,494.27011,FOODS,ADEREZOS,120.0,0,0,563.89955
...,...,...,...,...,...,...,...,...,...,...,...
33691,201902,21235,0.0,0.00000,0.00000,PC,PIEL1,200.0,0,1,0.00000
33692,201902,21236,0.0,0.00000,0.00000,PC,PIEL1,400.0,0,1,0.00000
33693,201902,21115,0.0,0.00000,0.00000,PC,DEOS,89.0,0,1,0.00000
33694,201902,20734,0.0,0.00000,0.00000,PC,CABELLO,400.0,0,1,0.00000


In [69]:
df[df["product_id"] == 20001]

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados,cluster,target
0,201701,20001,479.0,937.72717,934.77222,HC,ROPA LAVADO,3000.0,0,0,1303.35771
785,201702,20001,432.0,833.72187,798.01620,HC,ROPA LAVADO,3000.0,0,0,1069.96130
1566,201703,20001,509.0,1330.74697,1303.35771,HC,ROPA LAVADO,3000.0,0,0,1502.20132
2352,201704,20001,279.0,1132.94430,1069.96130,HC,ROPA LAVADO,3000.0,0,0,1520.06539
3136,201705,20001,701.0,1550.68936,1502.20132,HC,ROPA LAVADO,3000.0,0,0,1030.67391
3942,201706,20001,570.0,1575.82891,1520.06539,HC,ROPA LAVADO,3000.0,0,0,1267.39462
4765,201707,20001,381.0,1086.47101,1030.67391,HC,ROPA LAVADO,3000.0,0,0,1316.94604
5591,201708,20001,643.0,1289.66869,1267.39462,HC,ROPA LAVADO,3000.0,0,0,1439.75563
6438,201709,20001,381.0,1356.96103,1316.94604,HC,ROPA LAVADO,3000.0,0,0,1580.47401
7267,201710,20001,273.0,1441.60247,1439.75563,HC,ROPA LAVADO,3000.0,0,0,1049.38860


In [70]:
df["product_id"].nunique()

1296

In [71]:
df["periodo"].value_counts()

periodo
201701    1296
201702    1296
201901    1296
201812    1296
201811    1296
201810    1296
201809    1296
201808    1296
201807    1296
201806    1296
201805    1296
201804    1296
201803    1296
201802    1296
201801    1296
201712    1296
201711    1296
201710    1296
201709    1296
201708    1296
201707    1296
201706    1296
201705    1296
201704    1296
201703    1296
201902    1296
Name: count, dtype: int64

In [72]:
# Sort by `periodo` just to be sure
product_data = df.sort_values(["product_id", "periodo"])

In [73]:
product_data.shape, product_data.columns

((33696, 11),
 Index(['periodo', 'product_id', 'cust_request_qty', 'cust_request_tn', 'tn',
        'product_category', 'cat2', 'sku_size', 'plan_precios_cuidados',
        'cluster', 'target'],
       dtype='object'))

In [74]:
product_data.fillna(0, inplace=True)

In [75]:
# Using scikit-learn, ONE HOT ENCODE the categorical variables: product_category
product_data = pd.get_dummies(product_data, columns=["product_category", "cat2"])

# sort columns so that 'lag_tn' is the last column in the dataframe
product_data = product_data[[col for col in product_data.columns if col != 'target'] + ['target']]

In [76]:
# convert boolean columns to int
product_data = product_data.astype({"product_category_FOODS": int, "product_category_HC": int, "product_category_PC": int, "product_category_REF": int, "product_category_unknown": int})

In [77]:
# convert bool columns to int
values = [
    "CABELLO",
    "DEOS",
    "SOPAS Y CALDOS",
    "ROPA LAVADO",
    "PIEL2",
    "PIEL1",
    "HOGAR",
    "ADEREZOS",
    "VAJILLA",
    "ROPA ACONDICIONADOR",
    "OTROS",
    "DENTAL",
    "PROFESIONAL",
    "TE",
    "ROPA MANCHAS",
    "unknown"
]

for value in values:
    product_data = product_data.astype({f"cat2_{value}": int})
product_data.dtypes.sort_values()

periodo                       int64
cat2_VAJILLA                  int64
cat2_TE                       int64
cat2_SOPAS Y CALDOS           int64
cat2_ROPA MANCHAS             int64
cat2_ROPA LAVADO              int64
cat2_ROPA ACONDICIONADOR      int64
cat2_PROFESIONAL              int64
cat2_PIEL2                    int64
cat2_PIEL1                    int64
cat2_OTROS                    int64
cat2_HOGAR                    int64
cat2_DEOS                     int64
cat2_DENTAL                   int64
cat2_CABELLO                  int64
cat2_ADEREZOS                 int64
product_category_unknown      int64
product_category_REF          int64
product_category_PC           int64
product_category_HC           int64
product_category_FOODS        int64
cluster                       int64
plan_precios_cuidados         int64
product_id                    int64
cat2_unknown                  int64
sku_size                    float64
tn                          float64
cust_request_tn             

In [78]:
product_data.isna().sum().sum()

0

In [79]:
product_data.columns

Index(['periodo', 'product_id', 'cust_request_qty', 'cust_request_tn', 'tn',
       'sku_size', 'plan_precios_cuidados', 'cluster',
       'product_category_FOODS', 'product_category_HC', 'product_category_PC',
       'product_category_REF', 'product_category_unknown', 'cat2_ADEREZOS',
       'cat2_CABELLO', 'cat2_DENTAL', 'cat2_DEOS', 'cat2_HOGAR', 'cat2_OTROS',
       'cat2_PIEL1', 'cat2_PIEL2', 'cat2_PROFESIONAL',
       'cat2_ROPA ACONDICIONADOR', 'cat2_ROPA LAVADO', 'cat2_ROPA MANCHAS',
       'cat2_SOPAS Y CALDOS', 'cat2_TE', 'cat2_VAJILLA', 'cat2_unknown',
       'target'],
      dtype='object')

In [80]:
# Split the data into training and test sets
train_df = product_data[product_data["periodo"] <= 201811]
test_df = product_data[product_data["periodo"] <= 201812]
predict_df = product_data[product_data["periodo"] <= 201902]

In [81]:
# standarize the data
from sklearn.preprocessing import StandardScaler

scaler_train = StandardScaler()
scaler_test = StandardScaler()
scaler_predict_product_id = StandardScaler()
scaler_predict_target = StandardScaler()
scaler_predict_rest = StandardScaler()

rest = [col for col in predict_df.columns if col not in ["product_id", "target"]]

train_df[train_df.columns] = scaler_train.fit_transform(train_df[train_df.columns])
test_df[test_df.columns] = scaler_test.fit_transform(test_df[test_df.columns])
predict_df[rest] = scaler_predict_rest.fit_transform(predict_df[rest])
predict_df["target"] = scaler_predict_target.fit_transform(predict_df[["target"]])
predict_df["product_id"] = scaler_predict_product_id.fit_transform(predict_df[["product_id"]])

/tmp/ipykernel_117197/917755758.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[train_df.columns] = scaler_train.fit_transform(train_df[train_df.columns])
/tmp/ipykernel_117197/917755758.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[test_df.columns] = scaler_test.fit_transform(test_df[test_df.columns])


In [82]:
def train_model(n_layers, n_neurons, n_steps, X_train, y_train, X_test, y_test):
    # Train a single model where product_id is part of the input
    print(f"Training model with {n_layers} layers, {n_neurons} neurons, {n_steps} steps.")

    # Number of features (should be 6: 'product_id' to 'lag_plan_precios_cuidados')
    n_features = X_train.shape[2]

    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(50, activation="relu", input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mse")

    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Fit the model
    model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)
    return model

In [83]:
def make_predictions(n_steps, model, X_predict, y_predict):
    rows = []
    # for each product_id, generate a prediction for 201904
    for i, product in enumerate(predict_df["product_id"].unique()):
        print(
            f"Predicting 201904 for product {product} ({i+1}/{len(predict_df['product_id'].unique())}))"
        )

        this_predict_df = predict_df[predict_df["product_id"] == product]

        this_predict_df_array = this_predict_df.values

        n_features = X_predict.shape[2]

        # Prepare the input for prediction
        x_input = this_predict_df_array[-n_steps:, :-1]
        x_input = x_input.reshape((1, n_steps, n_features))

        # Make prediction
        yhat = model.predict(x_input, verbose=0)
        if yhat[0][0] < 1:
            yhat[0][0] = 0

        # Append to final output DataFrame
        rows.append(
            {
                "product_id": product,
                "target": yhat[0][0],
            }
        )
        final_output = pd.DataFrame(
            rows,
            columns=["product_id", "target"],
        )
        final_output = final_output.sort_values("product_id", ascending=True)
        timestamp = datetime.now().timestamp()
        final_output.to_csv(f"./output/output_lstm8_cluster_por_producto_{timestamp}.csv", index=False)
        print("//////////////////////////////////////")
        print(X_predict.shape, y_predict.shape, final_output.shape)
        print("//////////////////////////////////////")
        return final_output

In [84]:
def calculate_score(predictions):
    # revert the standarization for the output
    predictions["target"] = scaler_predict_target.inverse_transform(predictions[["target"]])
    predictions["product_id"] = scaler_predict_product_id.inverse_transform(predictions[["product_id"]])
    
    predictions["product_id"] = predictions["product_id"].astype(int)
    predictions.rename(columns={"target": "prediction"}, inplace=True)
    
    return calculate_error_2(predictions, 201904)


In [85]:
g_s_rows = []
for n_layers in NUMBER_OF_LAYERS:
    for n_neurons in NUMBER_OF_NEURONS:
        for n_steps in NUMBER_OF_STEPS:

            X_train, y_train = split_sequences(train_df.values, n_steps)
            X_test, y_test = split_sequences(test_df.values, n_steps)
            X_predict, y_predict = split_sequences(predict_df.values, n_steps)

            model = train_model(n_layers, n_neurons, n_steps, X_train, y_train, X_test, y_test)
            predictions = make_predictions(n_steps, model, X_predict, y_predict)
            score = calculate_score(predictions)

            g_s_rows.append(
                {
                    "n_layers": n_layers,
                    "n_neurons": n_neurons,
                    "n_steps": n_steps,
                    "score": score,
                }
            )

            g_s_df = pd.DataFrame(g_s_rows)
            g_s_df.to_csv(f"./output/grid_search_lstm8_cluster_productos_{n_layers}_{n_neurons[0]}_{n_steps}_{score}.csv", index=False)

Training model with 1 layers, [25, 25, 25, 25] neurons, 1 steps.


Epoch 1/200
932/932 [==============================] - 5s 4ms/step - loss: 0.1866 - val_loss: 0.1302
Epoch 2/200
932/932 [==============================] - 4s 4ms/step - loss: 0.1320 - val_loss: 0.1311
Epoch 3/200
932/932 [==============================] - 4s 4ms/step - loss: 0.1326 - val_loss: 0.1288
Epoch 4/200
932/932 [==============================] - 4s 4ms/step - loss: 0.1281 - val_loss: 0.1188
Epoch 5/200
932/932 [==============================] - 4s 4ms/step - loss: 0.1242 - val_loss: 0.1191
Epoch 6/200
932/932 [==============================] - 4s 4ms/step - loss: 0.1248 - val_loss: 0.1150
Epoch 7/200
932/932 [==============================] - 4s 4ms/step - loss: 0.1221 - val_loss: 0.1180
Epoch 8/200
932/932 [==============================] - 4s 4ms/step - loss: 0.1242 - val_loss: 0.1133
Epoch 9/200
932/932 [==============================] - 4s 4ms/step - loss: 0.1239 - val_loss: 0.1144
Epoch 10/200
932/932 [==============================] - 4s 4ms/step - loss: 0.1213 - val_lo

KeyboardInterrupt: 